In [ ]:
#pip install sentencepiece
#pip install --upgrade torch
#pip install --upgrade "optree>=0.13.0"
#ip install transformers torch


In [9]:
import transformers
print(transformers.__version__)  # Doit être ≥ 4.0

4.48.3


In [21]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from transformers import MarianMTModel, MarianTokenizer

import sentencepiece
import torch
import pandas as pd

In [6]:

model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

tweet = "I love artificial intelligence!"
tokens = tokenizer(tweet, return_tensors="pt")

# Traduction en français
translated_tokens = model.generate(**tokens, forced_bos_token_id=tokenizer.get_lang_id("fr"))
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

print(translated_text)


J’adore l’intelligence artificielle !


model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

In [25]:
column_names = ['class', 'id', 'date', 'query', 'name', 'tweet']
df = pd.read_csv("dataSet140.csv", sep=',', encoding='ISO-8859-1',names=column_names, nrows=100)
df.head()

,class,id,date,query,name,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [26]:
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_text(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    translated_tokens = model.generate(**tokens)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

df["tweet_fr"] = df["tweet"].astype(str).apply(translate_text)

print(df[["tweet", "tweet_fr"]].head())

df.to_csv("sentiment140_fr_2000.csv", index=False, encoding="utf-8")

c:\Users\bajaa\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


                                               tweet  \
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1  is upset that he can't update his Facebook by ...   
2  @Kenichan I dived many times for the ball. Man...   
3    my whole body feels itchy and like its on fire    
4  @nationwideclass no, it's not behaving at all....   

                                            tweet_fr  
0  @switchfoot http://twitpic.com/2y1zl - Awww, c...  
1  est contrarié qu'il ne puisse pas mettre à jou...  
2  @Kenichan J'ai plongé plusieurs fois pour la b...  
3  Tout mon corps se sent démangeant et comme sur...  
4  @nationwideclass non, ce n'est pas du tout se ...  
